In [6]:
import pandas as pd
import time

global final_reddit_topic_df
global top_post_df
global dict_topics



# Set up the app
reddit_post_df = pd.read_csv('500_Topics.csv')
sorted_reddit_post_df = reddit_post_df.sort_values(by=['Comment_Num'],ascending=False)
final_reddit_post_df = sorted_reddit_post_df.head(5)

X_train = final_reddit_post_df.Title

In [7]:
final_reddit_post_df

,Title,Score,Id,Url,Comment_Num,Created,Body,timestamp
336,IWTL how to respond when being made fun of,527,fbqduf,https://www.reddit.com/r/IWantToLearn/comments...,111,1.583074e+09,I’m a rather awkward person who doesn’t come u...,2020-03-01 20:22:23
1,IWTL how to reduce anxiety and worry of corona...,357,fhx4w5,https://www.reddit.com/r/IWantToLearn/comments...,107,1.584122e+09,Hey. As you can see from the title that i want...,2020-03-13 23:18:43
424,I want to learn how to make new friends as an ...,587,facr6i,https://www.reddit.com/r/IWantToLearn/comments...,104,1.582842e+09,"I am 21, I only have a small group of friends ...",2020-02-28 03:50:44
153,I want to learn how to help people through pan...,469,fet3ol,https://www.reddit.com/r/IWantToLearn/comments...,93,1.583603e+09,"So my friend has panic attacks, and I want to ...",2020-03-07 23:18:13
76,I want to learn how to have energy after work.,540,fg5mf1,https://www.reddit.com/r/IWantToLearn/comments...,80,1.583831e+09,"So I work about 7AM-6PM, with maybe an hour br...",2020-03-10 14:40:09


In [8]:
X_train

336           IWTL how to respond when being made fun of
1      IWTL how to reduce anxiety and worry of corona...
424    I want to learn how to make new friends as an ...
153    I want to learn how to help people through pan...
76        I want to learn how to have energy after work.
Name: Title, dtype: object

In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')

In [10]:
X_train_dtm = vectorizer.fit_transform(X_train)

In [11]:
vocab = np.array(vectorizer.get_feature_names())

In [12]:
vocab

array(['adult', 'anxiety', 'attacks', 'coronavirus', 'energy', 'friends',
       'fun', 'help', 'iwtl', 'learn', 'make', 'new', 'panic', 'people',
       'reduce', 'respond', 'want', 'work', 'worry'], dtype='<U11')

In [13]:
from sklearn import decomposition

num_topics = 5
num_top_words = 5
NMF_obj = decomposition.NMF(n_components=num_topics, random_state=1)

In [14]:
document_topics = NMF_obj.fit_transform(X_train_dtm)
document_topics.shape

(5, 5)

In [15]:
topics_word =[]
for topics in NMF_obj.components_:
    word_idx = np.argsort(topics)[::-1][0:num_top_words]
    topics_word.append([vocab[i] for i in word_idx])
topics_word

[['learn', 'want', 'new', 'friends', 'make'],
 ['worry', 'iwtl', 'anxiety', 'reduce', 'coronavirus'],
 ['help', 'attacks', 'people', 'panic', 'learn'],
 ['respond', 'fun', 'iwtl', 'worry', 'anxiety'],
 ['energy', 'work', 'learn', 'want', 'help']]

In [16]:
data_frame_topic = pd.DataFrame(document_topics,columns=topics_word)
data_frame_topic_inv = data_frame_topic.T
data_frame_topic

,learn,want,new,friends,make
,worry,iwtl,anxiety,reduce,coronavirus
,help,attacks,people,panic,learn
,respond,fun,iwtl,worry,anxiety
,energy,work,learn,want,help
0,0.000000,1.957369e-10,0.000000,1.421826,0.000000
1,0.000000,1.424040e+00,0.000000,0.000000,0.000000
2,1.354171,0.000000e+00,0.000000,0.000000,0.000000
3,0.000002,0.000000e+00,1.622840,0.000000,0.000303
4,0.000002,0.000000e+00,0.000012,0.000000,1.603916


In [17]:
data_frame_topic_inv[0]

learn    worry        help     respond  energy    0.000000e+00
want     iwtl         attacks  fun      work      1.957369e-10
new      anxiety      people   iwtl     learn     0.000000e+00
friends  reduce       panic    worry    want      1.421826e+00
make     coronavirus  learn    anxiety  help      0.000000e+00
Name: 0, dtype: float64

In [18]:
topics_series = []
for i in np.arange(data_frame_topic.shape[0]):
    topics_series.append(data_frame_topic_inv[i].argmax())

C:\Users\OM\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
topics_series

[('friends', 'reduce', 'panic', 'worry', 'want'),
 ('want', 'iwtl', 'attacks', 'fun', 'work'),
 ('learn', 'worry', 'help', 'respond', 'energy'),
 ('new', 'anxiety', 'people', 'iwtl', 'learn'),
 ('make', 'coronavirus', 'learn', 'anxiety', 'help')]

In [20]:
data_frame_topic['Dominant_Topic'] = topics_series
final_reddit_post_df['Dominant_Topic'] = topics_series
data_frame_topic
#final_reddit_post_df

C:\Users\OM\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,learn,want,new,friends,make,Dominant_Topic
,worry,iwtl,anxiety,reduce,coronavirus,
,help,attacks,people,panic,learn,
,respond,fun,iwtl,worry,anxiety,
,energy,work,learn,want,help,
0,0.000000,1.957369e-10,0.000000,1.421826,0.000000,"(friends, reduce, panic, worry, want)"
1,0.000000,1.424040e+00,0.000000,0.000000,0.000000,"(want, iwtl, attacks, fun, work)"
2,1.354171,0.000000e+00,0.000000,0.000000,0.000000,"(learn, worry, help, respond, energy)"
3,0.000002,0.000000e+00,1.622840,0.000000,0.000303,"(new, anxiety, people, iwtl, learn)"
4,0.000002,0.000000e+00,0.000012,0.000000,1.603916,"(make, coronavirus, learn, anxiety, help)"


In [21]:
final_reddit_post_df

,Title,Score,Id,Url,Comment_Num,Created,Body,timestamp,Dominant_Topic
336,IWTL how to respond when being made fun of,527,fbqduf,https://www.reddit.com/r/IWantToLearn/comments...,111,1.583074e+09,I’m a rather awkward person who doesn’t come u...,2020-03-01 20:22:23,"(friends, reduce, panic, worry, want)"
1,IWTL how to reduce anxiety and worry of corona...,357,fhx4w5,https://www.reddit.com/r/IWantToLearn/comments...,107,1.584122e+09,Hey. As you can see from the title that i want...,2020-03-13 23:18:43,"(want, iwtl, attacks, fun, work)"
424,I want to learn how to make new friends as an ...,587,facr6i,https://www.reddit.com/r/IWantToLearn/comments...,104,1.582842e+09,"I am 21, I only have a small group of friends ...",2020-02-28 03:50:44,"(learn, worry, help, respond, energy)"
153,I want to learn how to help people through pan...,469,fet3ol,https://www.reddit.com/r/IWantToLearn/comments...,93,1.583603e+09,"So my friend has panic attacks, and I want to ...",2020-03-07 23:18:13,"(new, anxiety, people, iwtl, learn)"
76,I want to learn how to have energy after work.,540,fg5mf1,https://www.reddit.com/r/IWantToLearn/comments...,80,1.583831e+09,"So I work about 7AM-6PM, with maybe an hour br...",2020-03-10 14:40:09,"(make, coronavirus, learn, anxiety, help)"


In [22]:
data_frame_topic_inv[0].argmax()

C:\Users\OM\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  """Entry point for launching an IPython kernel.


('friends', 'reduce', 'panic', 'worry', 'want')

In [24]:
uni_list = final_reddit_post_df.Dominant_Topic.unique()
uni_list

array([('friends', 'reduce', 'panic', 'worry', 'want'),
       ('want', 'iwtl', 'attacks', 'fun', 'work'),
       ('learn', 'worry', 'help', 'respond', 'energy'),
       ('new', 'anxiety', 'people', 'iwtl', 'learn'),
       ('make', 'coronavirus', 'learn', 'anxiety', 'help')], dtype=object)